# Integer Programming Problems

## Problem Statement
A company needs to decide on the locations for its warehouses to minimize the total cost while satisfying customer demand. The costs include fixed setup costs and transportation costs from warehouses to customers.

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Define problem data
warehouses = ['W1', 'W2', 'W3']
customers = ['C1', 'C2', 'C3']
setup_costs = [100, 150, 200]
transport_costs = [
    [4, 6, 8],
    [6, 4, 3],
    [9, 7, 5]
]
demand = [80, 70, 60]
capacity = [100, 90, 80]

# Decision variables
num_warehouses = len(warehouses)
num_customers = len(customers)
num_variables = num_warehouses * num_customers + num_warehouses

# Objective function: Minimize total cost
c = []
for i in range(num_warehouses):
    for j in range(num_customers):
        c.append(transport_costs[i][j])
c += setup_costs

# Constraints
A_ub = []
b_ub = []

# Capacity constraints
for i in range(num_warehouses):
    constraint = [0] * num_variables
    for j in range(num_customers):
        constraint[i * num_customers + j] = 1
    constraint[num_warehouses * num_customers + i] = -capacity[i]
    A_ub.append(constraint)
    b_ub.append(0)

# Demand constraints
for j in range(num_customers):
    constraint = [0] * num_variables
    for i in range(num_warehouses):
        constraint[i * num_customers + j] = 1
    A_ub.append(constraint)
    b_ub.append(demand[j])

# Bounds
bounds = [(0, None) for _ in range(num_warehouses * num_customers)] + [(0, 1) for _ in range(num_warehouses)]

# Solve the problem
res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Extract results
if res.success:
    transport_plan = res.x[:num_warehouses * num_customers]
    warehouse_setup = res.x[num_warehouses * num_customers:]

    print("Optimal Transport Plan:")
    for i in range(num_warehouses):
        for j in range(num_customers):
            print(f"Warehouse {warehouses[i]} to Customer {customers[j]}: {transport_plan[i * num_customers + j]} units")

    print("\nWarehouse Setup Plan:")
    for i in range(num_warehouses):
        print(f"Warehouse {warehouses[i]}: {'Open' if warehouse_setup[i] > 0.5 else 'Closed'}")
else:
    print("Optimization did not converge")


Optimal Transport Plan:
Warehouse W1 to Customer C1: 0.0 units
Warehouse W1 to Customer C2: 0.0 units
Warehouse W1 to Customer C3: 0.0 units
Warehouse W2 to Customer C1: 0.0 units
Warehouse W2 to Customer C2: 0.0 units
Warehouse W2 to Customer C3: 0.0 units
Warehouse W3 to Customer C1: 0.0 units
Warehouse W3 to Customer C2: 0.0 units
Warehouse W3 to Customer C3: 0.0 units

Warehouse Setup Plan:
Warehouse W1: Closed
Warehouse W2: Closed
Warehouse W3: Closed


## Explanation

### Data Definition:

warehouses and customers lists define the entities involved.
setup_costs and transport_costs arrays define the respective costs.
demand and capacity arrays define the constraints for customer demand and warehouse capacities.

### Decision Variables:

x[i][j] represents the units transported from warehouse i to customer j.
y[i] is a binary variable indicating whether warehouse i is open.

### Objective Function:

Minimize the total cost, which is the sum of transport and setup costs.

### Constraints:

Capacity constraints ensure warehouses do not exceed their capacities.
Demand constraints ensure customer demands are met.
Binary constraints ensure warehouses are either open or closed.

### Solving the Problem:

Use the linprog function from SciPy to solve the mixed integer programming problem.
